# Build Knowledge Graph from Tabular DNS Log Data
2/22/2025, Dave Sisk, https://github.com/davidcsisk, https://www.linkedin.com/in/davesisk-doctordatabase/

To use this notebook, you first need to login to Neo4j community site, create a free AuraDB instance (their cloud-based Neo4j offering) or empty your existing instance.  (I can't find a way to create a new database in a free AuraDB account, but you might be able to create a free 2nd instance, I'm not sure.) It will be paused once it's been idle for some number of days, but you can restart the AuraDB instance and it will still have the data (assuming you've already loaded it).  Start here...there are instructions along the way:
https://neo4j.com/docs/aura/classic/auradb/getting-started/create-database/

In [1]:
import pandas as pd

from dotenv import load_dotenv
import os

from neo4j import GraphDatabase

pd.set_option("display.max_columns", None)

- DNS log data from this URL: https://www.secrepo.com/maccdc2012/dns.log.gz
- Data description and columns names here: https://www.secrepo.com/Datasets%20Description/Network/dns.html
- Examine other data here as well: https://www.secrepo.com
- Helpful notebook: https://github.com/cyberdefendersprogram/MachineLearning/blob/master/Data_analysis/

In [2]:
# DNS log data from this URL: https://www.secrepo.com/maccdc2012/dns.log.gz
# Examine other data here as well: https://www.secrepo.com
# Helpful notebook: https://github.com/cyberdefendersprogram/MachineLearning/blob/master/Data_analysis/Network%20analysis/dns%20analysis.ipynb
df = pd.read_csv('dns.log.gz', compression='gzip', sep='\t', header=None)

# Rename columns based on this info: https://www.secrepo.com/Datasets%20Description/Network/dns.html
df = df.rename(columns={0:'ts', 1:'uid', 2:'src_ip', 3:'src_port', 4:'dest_ip', 5:'dest_port', 
                        6:'proto', 7:'port', 8:'query', 9:'qclass', 10:'qclass_name', 11:'qtype', 
                        12:'qtype_name', 13:'rcode', 14:'rcode_name', 15:'qr', 16:'aa', 17:'tc', 
                        18:'rd', 19:'z', 20:'answers', 21:'ttls', 22:'rejected'})


# Remove or replace invalid timestamp values
df["ts"] = pd.to_numeric(df["ts"], errors="coerce")  # Coerce invalid entries to NaN

# Ensure 'ts' is a datetime column before filling NaNs
df["ts"] = pd.to_datetime(df["ts"], unit="s", errors="coerce")

# Now safely replace NaNs with the Unix epoch timestamp
df.loc[df["ts"].isna(), "ts"] = pd.Timestamp("1970-01-01 00:00:00")


# Convert Unix timestamp to datetime
df["ts"] = pd.to_datetime(df["ts"], unit="s", errors="coerce")

# Remove any invalid rows
df = df.dropna(subset=["src_ip", "dest_ip", "query"])

In [3]:
df.sample(3)

,ts,uid,src_ip,src_port,dest_ip,dest_port,proto,port,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,qr,aa,tc,rd,z,answers,ttls,rejected
196999,2012-03-16 18:24:42.359999895,CicS02BhqphfDN2N6,10.10.117.209,53651,192.168.207.4,53,udp,41847,stats.norton.com,1,C_INTERNET,1,A,-,-,F,F,T,F,0,-,-,F
411441,2012-03-17 19:35:46.690000057,CutdkP3yDautUbBxFc,192.168.202.141,137,192.168.202.255,137,udp,57481,CONTENT.IE5,1,C_INTERNET,32,NB,-,-,F,F,T,F,1,-,-,F
107436,2012-03-16 17:00:03.779999971,CHwsqo48JzsgOOx5u5,10.10.117.210,19198,192.168.207.4,53,udp,9740,teredo.ipv6.microsoft.com,1,C_INTERNET,1,A,-,-,F,F,T,F,0,-,-,F


In [4]:
df.columns

Index(['ts', 'uid', 'src_ip', 'src_port', 'dest_ip', 'dest_port', 'proto',
       'port', 'query', 'qclass', 'qclass_name', 'qtype', 'qtype_name',
       'rcode', 'rcode_name', 'qr', 'aa', 'tc', 'rd', 'z', 'answers', 'ttls',
       'rejected'],
      dtype='object')

In [5]:
df.shape

(427935, 23)

In [6]:
# This is 427K rows of DNS data...maybe need to use a sample for test purposes.
df = df.head(10000)  # Get first 10K rows
#df = df.sample(10000) # Get a random sample of 10K rows

In [7]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = 'neo4j'

In [ ]:
from neo4j import GraphDatabase

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def create_dns_graph(tx, batch):
    query = """
    UNWIND $batch AS row
    MERGE (src:IP {address: row.src_ip})
    MERGE (dest:IP {address: row.dest_ip})
    MERGE (domain:Domain {name: row.query})
    MERGE (proto:Protocol {name: row.proto})
    MERGE (sport:Port {number: row.src_port})
    MERGE (dport:Port {number: row.dest_port})

    MERGE (src)-[:QUERIED {timestamp: row.ts, uid: row.uid, qtype: row.qtype_name, rcode: row.rcode_name}]->(domain)
    MERGE (src)-[:USES_PORT]->(sport)
    MERGE (dest)-[:USES_PORT]->(dport)
    MERGE (src)-[:USES_PROTOCOL]->(proto)

    FOREACH (answer IN row.answers | 
        MERGE (ans_ip:IP {address: answer})
        MERGE (domain)-[:HAS_ANSWER]->(ans_ip)
    )
    """
    tx.run(query, batch=batch)  # Pass batch correctly

# Load data in batches
BATCH_SIZE = 1000  # Adjust batch size as needed

with driver.session(database=NEO4J_DATABASE) as session:
    for i in range(0, len(df), BATCH_SIZE):
        batch = df.iloc[i : i + BATCH_SIZE].copy()  # Avoid SettingWithCopyWarning

        # Ensure answers column is a list
        batch["answers"] = batch["answers"].apply(lambda x: x.split(",") if isinstance(x, str) else [])

        # Convert batch to list of dictionaries
        batch_dict = batch.to_dict(orient="records")

        #session.write_transaction(create_dns_graph, batch_dict)
        session.execute_write(create_dns_graph, batch_dict)

# Close Neo4j connection
driver.close()

In [9]:
# Helper function for Neo4J queries
from neo4j import GraphDatabase

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
TARGET_DB='neo4j'
# Function to execute Cypher queries and return results
def run_query(query, params={}):
    with driver.session(database=TARGET_DB) as session:
        result = session.run(query, params)
        return [record for record in result]

# Test connection
print("Neo4j connection established.")

Neo4j connection established.


In [10]:
# Is there data there?
query = """
MATCH (n) RETURN n LIMIT 10
"""

run_query(query)

[<Record n=<Node element_id='4:015e1c75-ac37-496e-85a1-1fb0d86f4b10:0' labels=frozenset({'IP'}) properties={'address': '192.168.202.100'}>>,
 <Record n=<Node element_id='4:015e1c75-ac37-496e-85a1-1fb0d86f4b10:1' labels=frozenset({'IP'}) properties={'address': '192.168.202.76'}>>,
 <Record n=<Node element_id='4:015e1c75-ac37-496e-85a1-1fb0d86f4b10:2' labels=frozenset({'IP'}) properties={'address': '192.168.202.89'}>>,
 <Record n=<Node element_id='4:015e1c75-ac37-496e-85a1-1fb0d86f4b10:3' labels=frozenset({'IP'}) properties={'address': '192.168.202.85'}>>,
 <Record n=<Node element_id='4:015e1c75-ac37-496e-85a1-1fb0d86f4b10:4' labels=frozenset({'IP'}) properties={'address': '192.168.202.102'}>>,
 <Record n=<Node element_id='4:015e1c75-ac37-496e-85a1-1fb0d86f4b10:5' labels=frozenset({'IP'}) properties={'address': '192.168.202.93'}>>,
 <Record n=<Node element_id='4:015e1c75-ac37-496e-85a1-1fb0d86f4b10:6' labels=frozenset({'IP'}) properties={'address': '192.168.202.97'}>>,
 <Record n=<Node e

In [11]:
# Top 10 Most Queried Domains
query = """
MATCH (d:Domain)<-[q:QUERIED]-()
RETURN d.name AS domain, COUNT(q) AS query_count
ORDER BY query_count DESC
LIMIT 10
"""
run_query(query)

[<Record domain='44.206.168.192.in-addr.arpa' query_count=617>,
 <Record domain='www.apple.com' query_count=576>,
 <Record domain='WPAD' query_count=534>,
 <Record domain='ARMMF.ADOBE.COM' query_count=420>,
 <Record domain='ISATAP' query_count=380>,
 <Record domain='creativecommons.org' query_count=342>,
 <Record domain='www.dokuwiki.org' query_count=324>,
 <Record domain='www.php.net' query_count=308>,
 <Record domain='*\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00' query_count=270>,
 <Record domain='HPE8AA67' query_count=249>]

In [12]:
# Most Active Source IPs
query = """
MATCH (src:IP)-[q:QUERIED]->(d:Domain)
RETURN src.address AS source_ip, COUNT(q) AS queries
ORDER BY queries DESC
LIMIT 10
"""
run_query(query)

[<Record source_ip='192.168.202.102' queries=965>,
 <Record source_ip='192.168.202.93' queries=914>,
 <Record source_ip='192.168.202.97' queries=901>,
 <Record source_ip='192.168.202.103' queries=840>,
 <Record source_ip='192.168.203.62' queries=742>,
 <Record source_ip='192.168.202.76' queries=652>,
 <Record source_ip='192.168.202.83' queries=641>,
 <Record source_ip='192.168.202.79' queries=477>,
 <Record source_ip='192.168.202.89' queries=460>,
 <Record source_ip='192.168.203.61' queries=374>]

In [13]:
# Most Common Response IPs
query = """
MATCH (d:Domain)-[:HAS_ANSWER]->(ip:IP)
RETURN ip.address AS answer_ip, COUNT(*) AS occurrences
ORDER BY occurrences DESC
LIMIT 10
"""
run_query(query)

[<Record answer_ip='-' occurrences=579>,
 <Record answer_ip='110.209.6.25' occurrences=2>,
 <Record answer_ip='c02gn35udjwr._sftp-ssh._tcp.local' occurrences=2>,
 <Record answer_ip='enigma.local' occurrences=1>,
 <Record answer_ip='192.168.208.18' occurrences=1>,
 <Record answer_ip='c02gn35udjwr.local' occurrences=1>,
 <Record answer_ip='sds-macbook-pro.local' occurrences=1>,
 <Record answer_ip='dnsmasq-2.45' occurrences=1>,
 <Record answer_ip='Microsoft DNS 6.0.6002 (17724655)' occurrences=1>,
 <Record answer_ip='2001:dbb:c18:202:f2de:f1ff:fe9b:ad6a' occurrences=1>]

In [14]:
# Queries That Were Rejected
query = """
MATCH (src:IP)-[q:QUERIED {rcode: 'NXDOMAIN'}]->(d:Domain)
RETURN d.name AS rejected_domain, COUNT(q) AS attempts
ORDER BY attempts DESC
LIMIT 10
"""
run_query(query)

[<Record rejected_domain='44.206.168.192.in-addr.arpa' attempts=617>,
 <Record rejected_domain='creativecommons.org' attempts=342>,
 <Record rejected_domain='www.dokuwiki.org' attempts=324>,
 <Record rejected_domain='www.php.net' attempts=308>,
 <Record rejected_domain='validator.w3.org' attempts=184>,
 <Record rejected_domain='dokuwiki.org' attempts=184>,
 <Record rejected_domain='jigsaw.w3.org' attempts=184>,
 <Record rejected_domain='www.stopbadware.org' attempts=102>,
 <Record rejected_domain='safebrowsing.clients.google.com' attempts=80>,
 <Record rejected_domain='smolt.openfiler.com' attempts=72>]

In [15]:
# DNS Requests Using a Specific Protocol
query = """
MATCH (src:IP)-[:USES_PROTOCOL]->(proto:Protocol)
WHERE proto.name = 'udp'
RETURN src.address AS source_ip, COUNT(*) AS udp_queries
ORDER BY udp_queries DESC
LIMIT 10
"""
udp_requests = run_query(query)
print(udp_requests)


[<Record source_ip='192.168.202.76' udp_queries=1>, <Record source_ip='192.168.202.89' udp_queries=1>, <Record source_ip='192.168.202.85' udp_queries=1>, <Record source_ip='192.168.202.102' udp_queries=1>, <Record source_ip='192.168.202.93' udp_queries=1>, <Record source_ip='192.168.202.97' udp_queries=1>, <Record source_ip='fe80::ba8d:12ff:fe53:a8d8' udp_queries=1>, <Record source_ip='192.168.202.71' udp_queries=1>, <Record source_ip='192.168.202.77' udp_queries=1>, <Record source_ip='192.168.202.100' udp_queries=1>]


In [17]:
# All Connections for a Specific Domain
domain_name = "time.apple.com"  # Change this to a domain of interest

query = """
MATCH (src:IP)-[q:QUERIED]->(d:Domain {name: $domain_name})-[:HAS_ANSWER]->(ip:IP)
RETURN src.address AS source_ip, ip.address AS answer_ip, q.timestamp AS timestamp
ORDER BY timestamp DESC
LIMIT 20
"""
connections = run_query(query, {"domain_name": domain_name})
print(connections)


[<Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.DateTime(2012, 3, 16, 13, 7, 13, 240000010)>, <Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.DateTime(2012, 3, 16, 13, 7, 4, 220000029)>, <Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.DateTime(2012, 3, 16, 13, 7, 1, 220000029)>, <Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.DateTime(2012, 3, 16, 13, 7, 0, 220000029)>, <Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.DateTime(2012, 3, 16, 13, 6, 43, 250000000)>, <Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.DateTime(2012, 3, 16, 13, 6, 34, 230000019)>, <Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.DateTime(2012, 3, 16, 13, 6, 31, 220000029)>, <Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.DateTime(2012, 3, 16, 13, 6, 30, 220000029)>, <Record source_ip='192.168.202.93' answer_ip='-' timestamp=neo4j.time.Date